In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [81]:
# Resources:

# https://stackoverflow.com/questions/52495105/wrong-colors-in-result-pyramid-blending-using-opencv-and-python
# https://theailearner.com/tag/image-pyramids-opencv-python/

In [88]:
def pyramid(jetImage, cloudImage, maskImage, levels=1):
#     A == jetImage
#     B = cloudImage
#     m = maskImage
#     jetImage, , 
    
    # Keep mask values between 0-1
    maskImage = maskImage/255.0

    # Copy the input images and mask
    jet = jetImage[:,:,:] # GA
    cloud = cloudImage[:,:,:] # GB
    mask = maskImage[:,:,:] # GM

    # Initialize Gaussian pyramids
    gaussianPyramidJet   = [jet] # gpA
    gaussianPyramidCloud = [cloud] #gpB
    gaussianPyramidMask  = [mask] #gpM

    # Build Gaussian Pyramids
    for i in range(levels):
        jet = cv2.pyrDown(jet)
        cloud = cv2.pyrDown(cloud)
        mask = cv2.pyrDown(mask)

        gaussianPyramidJet.append(np.float32(jet))
        gaussianPyramidCloud.append(np.float32(cloud))
        gaussianPyramidMask.append(np.float32(mask))
    #end-for

    # Initialize Laplacian pyramids
    laplacianPyramidJet = [gaussianPyramidJet[levels - 1]] # lpA
    laplacianPyramidCloud = [gaussianPyramidCloud[levels - 1]] #lpB
    laplacianPyramidMask = [gaussianPyramidMask[levels - 1]] #gpMr

    # Build Laplacian pyramids
    for i in range(levels - 1, 0, -1):
        size = (gaussianPyramidJet[i - 1].shape[1], gaussianPyramidJet[i - 1].shape[0])

        LA = np.subtract(gaussianPyramidJet[i - 1], cv2.pyrUp(gaussianPyramidJet[i], dstsize=size))
        LB = np.subtract(gaussianPyramidCloud[i - 1], cv2.pyrUp(gaussianPyramidCloud[i], dstsize=size))

        laplacianPyramidJet.append(LA)
        laplacianPyramidCloud.append(LB)

        laplacianPyramidMask.append(gaussianPyramidMask[i - 1])

    # Image Blending: employing Laplacian pyramids
    # https://www.cs.toronto.edu/~mangas/teaching/320/assignments/a3/
    LS = []
    for l_Jet, l_Cloud, l_Mask in zip(laplacianPyramidJet, laplacianPyramidCloud, laplacianPyramidMask):
        ls = l_Jet * l_Mask + l_Cloud * (1.0 - l_Mask)
        LS.append(ls)
    #end-for

    # reconstruct the blended image
    ls = LS[0]
    for i in range(1, levels):
        size = (LS[i].shape[1], LS[i].shape[0])
        ls = cv2.add(cv2.pyrUp(ls, dstsize=size), np.float32(LS[i]))

        # Threshold controlling:
        ls[ls > 255] = 255
        ls[ls < 0]   = 0
    #end-for
    return ls
#end-def

In [90]:
jetImage   = cv2.imread('jet.jpg')
cloudImage = cv2.imread('cloud.jpg')
maskImage  = cv2.imread('mask.jpg')

for i in range(21):
    output = pyramid(jetImage, cloudImage, maskImage, i)
    cv2.imwrite(f'outputs/result_{i}.jpg', output)
#end-for